In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import cv2
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline

Using TensorFlow backend.


In [2]:
import ali

class MyConfig(Config):
    NUM_CLASSES = 6
    NAME = 'ali'
    BACKBONE = 'resnet50'

config = MyConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                18
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

In [3]:
DATA_DIR = "./*/"

dataset = ali.AliDataset()
dataset.load_ali(DATA_DIR, 'restricted')
# Must call before using the dataset
dataset.prepare()

print("Train Image Count: {}".format(len(dataset.image_ids)))
print("Train Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
Train Image Count: 977
Train Class Count: 6
  0. BG                                                
  1. 铁壳打火机                                             
  2. 黑钉打火机                                             
  3. 刀具                                                
  4. 电源和电池                                             
  5. 剪刀                                                


In [4]:
def split(dataset, image_ids):
    s = ali.AliDataset()
    s.class_info = dataset.class_info
    s.class_ids = dataset.class_ids
    for i in image_ids:
        s.add_image('ali', image_id=i, path=dataset.image_info[i]['path'], 
                       annotations=dataset.image_info[i]['annotations'], 
                       height=dataset.image_info[i]['height'], 
                       width=dataset.image_info[i]['width'])
    return s

In [5]:
image_ids = dataset.image_ids.copy()
random.seed(42)
random.shuffle(image_ids)
val_size = 200
image_val = image_ids[:val_size]
image_train = image_ids[val_size:]

In [6]:
dataset_train = split(dataset, image_train)
dataset_train.prepare()
dataset_val = split(dataset, image_val)
dataset_val.prepare()

In [7]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

In [8]:
NUM_EPOCHS = 1

# Train Mask-RCNN Model 
import warnings 
warnings.filterwarnings("ignore")
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/git/Mask_RCNN/samples/jinnan2/ali20190306T1404/mask_rcnn_ali_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_m

Process ForkPoolWorker-25:
Process ForkPoolWorker-24:
Process ForkPoolWorker-11:
Process ForkPoolWorker-27:
Process ForkPoolWorker-26:
Process ForkPoolWorker-9:
Process ForkPoolWorker-28:
Process ForkPoolWorker-21:
Process ForkPoolWorker-19:
Process ForkPoolWorker-23:
Process ForkPoolWorker-30:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-10:
Process ForkPoolWorker-29:
Process ForkPoolWorker-20:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-14:
Process ForkPoolWorker-3:
Process ForkPoolWorker-16:
Process ForkPoolWorker-18:
Process ForkPoolWorker-2:
Process ForkPoolWorker-31:
Process ForkPoolWorker-15:
Process ForkPoolWorker-13:
Process ForkPoolWorker-6:
Process ForkPoolWorker-32:
Process ForkPoolWorker-22:
Traceback (most recent call last):
Process ForkPoolWorker-12:
Process ForkPoolWorker-4:
Process ForkPoolWorker-17:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Trace

  File "/usr/local/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/local/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._

  File "/usr/local/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/local/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/local/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()


KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")